In [1]:
import pickle
import numpy as np

In [2]:
file = open('data.pickle', 'rb')
data_dict = pickle.load(file)
data = data_dict['data']
labels = data_dict['labels']

In [4]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.activations import relu, softmax

In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation=relu)(x)
x = Dense(10, activation=softmax)(x)

87910968/87910968 [==============================] - 61s 1us/step


In [7]:
numbers_model = Model(inputs=base_model.input, outputs=x)

In [6]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, None, None, 32)       96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, None, None, 32)       0         ['batch_normalizati

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
numbers_model.compile(optimizer=Adam(),loss=categorical_crossentropy, metrics=[Accuracy()] )

In [10]:
numbers_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, None, None, 32)       96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, None, None, 32)       0         ['batch_normalization[0][0

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
y = to_categorical(labels, num_classes=10)

In [13]:
X = data / 255.0

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

In [16]:
numbers_model.fit(X_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
3/3 [==============================] - 4s 1s/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 2/25
3/3 [==============================] - 5s 2s/step - loss: 0.0026 - accuracy: 0.0000e+00
Epoch 3/25
3/3 [==============================] - 5s 2s/step - loss: 0.0020 - accuracy: 0.0000e+00
Epoch 4/25
3/3 [==============================] - 6s 2s/step - loss: 0.0016 - accuracy: 0.0000e+00
Epoch 5/25
3/3 [==============================] - 5s 2s/step - loss: 0.0014 - accuracy: 0.0000e+00
Epoch 6/25
3/3 [==============================] - 6s 2s/step - loss: 0.0013 - accuracy: 0.0000e+00
Epoch 7/25
3/3 [==============================] - 6s 2s/step - loss: 0.0012 - accuracy: 0.0012
Epoch 8/25
3/3 [==============================] - 6s 2s/step - loss: 0.0011 - accuracy: 0.0000e+00
Epoch 9/25
3/3 [==============================] - 6s 2s/step - loss: 0.0011 - accuracy: 0.0000e+00
Epoch 10/25
3/3 [==============================] - 6s 2s/step - loss: 0.0010 - accuracy: 0.0000e+00
Epoch 11/25
3

In [17]:
y_pred = numbers_model.predict(X_test)

1/1 [==============================] - 2s 2s/step


In [19]:
y_pred_classes = y_pred.argmax(axis=1)

In [20]:
y_pred_classes

array([5, 6, 6, 0, 3, 1, 8, 5, 2, 9, 1, 3, 3, 9, 0, 7, 4, 7, 2, 0, 8, 4])

In [22]:
y_test_classes = y_test.argmax(axis=1)

In [23]:
y_test_classes

array([5, 6, 6, 0, 3, 1, 8, 5, 2, 9, 1, 3, 3, 9, 0, 7, 4, 7, 2, 0, 8, 4])

In [24]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred_classes, y_test_classes))

1.0


In [2]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

2023-08-23 17:04:53.254256: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 17:04:53.553339: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 17:04:53.554962: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 17:04:55.807691: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
img = image.load_img('./test.png')
img_array = image.img_to_array(img)
input = img_array / 255.0
plt.imshow(img_array)


NameError: name 'image' is not defined

In [36]:
y_new = numbers_model.predict(np.asarray([input]))
print(y_new.argmax(axis=1)[0])

1/1 [==============================] - 0s 65ms/step
2


In [37]:
file = open('model.p', 'wb')
pickle.dump({'model' : numbers_model}, file)
file.close()